In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
import cv2
import time
import os

def cv_resize_pad(img,desired_size):
    osize = img.shape[:2]
    ratio = float(desired_size)/max(osize)
    nsize = tuple([int(x*ratio) for x in osize])
    
    img = cv2.resize(img, (nsize[1],nsize[0]))
    dw = desired_size - nsize[1]
    dh = desired_size - nsize[0]

    top,bottom = dh//2,dh-(dh//2)
    left,right = dw//2,dw-(dw//2)

    color = [0,0,0]
    return cv2.copyMakeBorder(img,top,bottom,left,right,cv2.BORDER_CONSTANT,value=color)

def generate_seqs(images,data_desc):
    idx = []
    runn_idx = 0
    img_seqs = []
    labels = []
    label = None
    tid = 0 
    for _,row in data_desc.iterrows():
        if(tid != row['trackid']):
            if(len(idx)!=0):
                print(row)
                idx = list(map(lambda x: x+runn_idx,idx))
                img_seqs.append(np.array(images[idx]))
                labels.append(label)
                runn_idx = runn_idx + len(idx)
            
            tid = row['trackid']
            idx = [row['framenr']-2] #TODO
        else:
            idx.append(row['framenr']-2)
        label = row['class']
    return img_seqs,labels

def load_images(data_dir,resize=None):
    dir_list = ['boat/','nature/']
    img_list = []
    label_list= []
    shapes = []
    for sub_dir in dir_list:
        filenames = os.listdir(os.path.join(data_dir,sub_dir))
        for filename in filenames:
            if filename[-3:]=='png':
                try:
                    img_raw = cv2.imread(data_dir+sub_dir+filename,cv2.IMREAD_COLOR)
                    img_list.append(img_raw)
                    label_list.append(sub_dir[:-1])
                    shapes.append((img_raw.shape[0],img_raw.shape[1]))
                    time.sleep(0.1)

                except:
                    print('Did not load: ',i)
                    pass

    shapes = np.array(shapes)
    if(resize==None):
        resize = np.int8((np.mean(shapes[:,0])+np.mean(shapes[:,1]))/2.0)
    res_imgs = []
    for img in img_list:
        nimg = cv_resize_pad(img,resize)
        # More augmentation 
        res_imgs.append(nimg)

    images = np.array(res_imgs)
    labels = np.array(label_list)
    return images, labels
  
data_dir = 'Bodensee/'
train_dir = data_dir +'train/'
test_dir = data_dir +'test/'
train_imgs,train_labels = load_images(train_dir)
test_imgs,test_labels = load_images(test_dir,resize=train_imgs.shape[2])
train_dataset = tf.data.Dataset.from_tensor_slices((train_imgs,train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_imgs,test_labels))

In [4]:
!ls

6068.png                 bird.jpg                 testing.py
Bodensee                 custom_augmentation.py
Untitled.ipynb           ship-detection-challenge


In [ ]:
cd Bodensee/train